#### 1 Introduction

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# Base URL para los chunks
base_url = "https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_"

In [ ]:
# Lista para almacenar los datos de todos los chunks
all_data = []

In [ ]:
# Iterar sobre las 10 páginas
for i in range(1, 11):
    # Construir la URL para cada chunk
    url = f"{base_url}{i}.html"
    print(f"Scraping {url}...")  # Opcional: para monitorear el progreso

    # Hacer la solicitud HTTP
    response = requests.get(url)

    # Parsear el contenido HTML
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontrar la tabla
    table = soup.find('table')

    # Extraer los encabezados
    headers = [header.text.strip() for header in table.find_all('th')]

    # Extraer las filas de la tabla
    rows = []
    for row in table.find_all('tr'):
        cells = [cell.text.strip() for cell in row.find_all('td')]
        if cells:
            rows.append(cells)

    # Crear un DataFrame temporal
    df = pd.DataFrame(rows, columns=headers)

    # Agregar el DataFrame temporal a la lista de todos los datos
    all_data.append(df)

Scraping https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_1.html...
Scraping https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_2.html...
Scraping https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_3.html...
Scraping https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_4.html...
Scraping https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_5.html...
Scraping https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_6.html...
Scraping https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_7.html...
Scraping https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_8.html...
Scraping https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_9.html...
Scraping https://ignaciomsarmiento.github.io/GEIH2018_sample/pages/geih_page_10.html...


In [ ]:
# Combinar todos los DataFrames en uno solo
final_data = pd.concat(all_data, ignore_index=True)

In [ ]:
display(final_data)

,,directorio,secuencia_p,orden,clase,dominio,mes,estrato1,sex,age,...,y_viaticos_m,y_accidentes_m,y_salarySec_m,y_ingLab_m_ha,y_gananciaNeta_m,y_gananciaNetaAgro_m,y_gananciaIndep_m,y_gananciaIndep_m_hu,y_total_m,y_total_m_ha
0,1,4514331,1,2,1,BOGOTA,1,2,0,29,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,2,4514331,1,1,1,BOGOTA,1,2,1,36,...,NA,NA,NA,8404.3203125,NA,NA,NA,NA,1620833.25,8404.3203125
2,3,4514332,1,4,1,BOGOTA,1,2,1,4,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,4,4514332,1,3,1,BOGOTA,1,2,1,7,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,5,4514332,1,1,1,BOGOTA,1,2,0,32,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32172,3213,4804454,1,2,1,BOGOTA,12,2,0,24,...,NA,NA,NA,3345.5556640625,NA,NA,NA,NA,1003666.6875,3345.5556640625
32173,3214,4804455,1,1,1,BOGOTA,12,3,0,36,...,5e+05,NA,NA,25958.333984375,NA,NA,NA,NA,6675000,25958.333984375
32174,3215,4804455,1,2,1,BOGOTA,12,3,1,41,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
32175,3216,4804455,1,3,1,BOGOTA,12,3,1,8,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


# Introducción

En este trabajo práctico, nuestro objetivo es construir un modelo de predicción de ingresos por hora individual:

$$
w = f(X) + u
$$

donde $w$ representa el ingreso por hora, y $X$ es una matriz de variables explicativas/predictoras potenciales. Nos centraremos en un modelo lineal de la forma:

$$
\ln(w) = \beta_0 + \beta_1 X_1 + \dots + \beta_p X_p + u
$$

Para evaluar la capacidad predictiva del modelo, dividiremos el conjunto de datos en dos subconjuntos: una parte denominada "live-data "para entrenar el modelo y otra aprte denominada "unseen-data" evaluar su desempeño, siguiendo el método **train-test split** de la librería **SKLearn**. El uso de particiones como 70%-30% o 80%-20% nos permitirá balancear la cantidad de datos destinados al entrenamiento y la evaluación, asegurando a su vez reproducibilidad mediante un **random state**. Este enfoque es particularmente relevante en problemas donde el tamaño de la muestra impacta la precisión del modelo. Al dividir la muestra perderemos eficiencia pero ganamos por el lado de estimar directamente el error de pérdida. Además, incorporaremos técnicas de remuestreo como **bootstrap**, que permiten generar múltiples subconjuntos de datos mediante muestreo aleatorio con reemplazo. Estas técnicas son útiles para estimar la incertidumbre en las predicciones y evaluar la robustez del modelo frente a diferentes particiones de datos.


Para este análisis, utilizaremos datos de Bogotá obtenidos del informe de 2018 *Medición de Pobreza Monetaria y Desigualdad*, basado en la Gran Encuesta Integrada de Hogares (GEIH). Este conjunto de datos proporciona información detallada sobre empleo, ingresos, educación y características demográficas, lo que lo hace adecuado para el análisis.

En las siguientes secciones, describiremos el proceso de obtención y limpieza de los datos, realizaremos un análisis exploratorio y evaluaremos el poder predictivo de distintas especificaciones del modelo. Además, exploraremos cómo las técnicas de remuestreo como bootstrap contribuyen a una mejor estimación del error de predicción fuera de muestra.

### 1.1 General Instructions


In [ ]:
#generar una copia de la base de datos con copy
df_1 = final_data.copy()

In [5]:
#Filtro el df con edad +18 y que estén empleados

df_2 = df_1[(df_1["age"]>=18)&(df_1["ocu"]==1)].copy()

In [ ]:
df_2.shape

(16542, 178)

In [ ]:
#Variable y_ingLab_m_ha debe ser la dependiente. Es el salario por hora.

df_2['y_ingLab_m_ha'].agg(['mean', 'median', 'std', 'min', 'max'])

print(df_1['y_ingLab_m_ha'].isna().sum())
print((df_1['y_ingLab_m_ha'] == 0).sum())


22213
0


## 3 Predicting Wages


In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
df_2.head()

,Unnamed: 0,directorio,secuencia_p,orden,clase,dominio,mes,estrato1,sex,age,...,y_viaticos_m,y_accidentes_m,y_salarySec_m,y_ingLab_m_ha,y_gananciaNeta_m,y_gananciaNetaAgro_m,y_gananciaIndep_m,y_gananciaIndep_m_hu,y_total_m,y_total_m_ha
1,2,4514331,1,1,1,BOGOTA,1,2,1,36,...,NaN,NaN,NaN,8404.320312,NaN,NaN,NaN,NaN,1620833.250,8404.320312
8,9,4514333,1,1,1,BOGOTA,1,2,1,51,...,NaN,NaN,NaN,6703.342773,NaN,NaN,NaN,NaN,1378973.375,6703.342773
13,14,4514334,1,1,1,BOGOTA,1,2,1,45,...,NaN,NaN,NaN,3680.654053,NaN,NaN,NaN,NaN,883357.000,3680.654053
15,16,4514335,1,1,1,BOGOTA,1,2,1,61,...,NaN,NaN,NaN,4083.333252,NaN,NaN,NaN,NaN,700000.000,4083.333252
21,22,4514338,1,2,1,BOGOTA,1,2,0,35,...,NaN,NaN,NaN,9437.847656,NaN,NaN,NaN,NaN,1941500.125,9437.847656


In [ ]:
df_2.rename(columns={'y_ingLab_m_ha': 'w'} , inplace=True)  # renombro dependiente
df_2.drop(columns=['Unnamed: 0', 'directorio', 'fex_c', 'fex_dpto', 'orden' ], inplace=True)


# saco predictores que creo no son relevantes
cols_to_drop = [col for col in df_2.columns if col.startswith('y_') | col.startswith('p') | col.startswith('i')]

# Elimine esas columnas
df_3 = df_2.drop(columns=cols_to_drop )



In [ ]:
df_3.head()

,secuencia_p,clase,dominio,mes,estrato1,sex,age,oficio,relab,hoursWorkUsual,...,cotPension,wap,ocu,dsi,totalHoursWorked,formal,cuentaPropia,microEmpresa,sizeFirm,w
1,1,1,BOGOTA,1,2,1,36,39.0,2.0,45.0,...,1.0,1,1,0,45.0,1.0,0,0.0,5.0,8404.320312
8,1,1,BOGOTA,1,2,1,51,85.0,1.0,48.0,...,1.0,1,1,0,48.0,1.0,0,0.0,3.0,6703.342773
13,1,1,BOGOTA,1,2,1,45,45.0,1.0,56.0,...,1.0,1,1,0,56.0,1.0,0,1.0,2.0,3680.654053
15,1,1,BOGOTA,1,2,1,61,53.0,1.0,40.0,...,2.0,1,1,0,40.0,0.0,0,0.0,4.0,4083.333252
21,1,1,BOGOTA,1,2,0,35,13.0,2.0,48.0,...,1.0,1,1,0,48.0,1.0,0,0.0,5.0,9437.847656


In [ ]:
X = df_3.loc[:, 'secuencia_p':'sizeFirm'] #tomo predictores

y = df_3.loc[:, 'w'] # tomo variable dependiente

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y,
                                        test_size=0.3,
                                        train_size=0.7,
                                        random_state = 123
                                    )  # split the sample into two